### Topological Sort

In [3]:
def topologicalSort(G):
    visited = set()
    post_order = []
    
    def dfs(node):
        visited.add(node)
        for neighbor in G[node]:
            if neighbor not in visited:
                dfs(neighbor)
        post_order.append(node)
        
    for node in G:
        if node not in visited:
            dfs(node)
            
    sorted_vertices = list(reversed(post_order))
    return sorted_vertices

In [4]:
graph = {
    'A': ['B', 'C'],
    'B': ['C', 'D'],
    'C': ['D'],
    'D': []
}

sorted_nodes = topologicalSort(graph)
print("Sorted vertices: ", sorted_nodes)

Sorted vertices:  ['A', 'B', 'C', 'D']


### Strongly Connected Components

#### Algorithm

##### EasySCC(G)
```
for each vertex v:
    run explore(v) to determine vertices reachable from v
for each vertex v:
    find the u reachable from v that can also reach v
these are the SCCs
```


In [3]:
def explore(v, visited, adj_list, stack):
    visited.add(v)
    for neighbor in adj_list[v]:
        if neighbor not in visited:
            explore(neighbor, visited, adj_list, stack)
    stack.append(v)
    
def transpose(graph):
    transposed = {}
    for vertex in graph:
        transposed[vertex] = []
    for vertex in graph:
        for neighbor in graph[vertex]:
            transposed[neighbor].append(vertex)
    return transposed

def find_scc(graph):
    visited = set()
    stack = []
    for vertex in graph:
        if vertex not in visited:
            explore(vertex, visited, graph, stack)
    
    transposed_graph = transpose(graph)
    
    visited.clear()
    scc_list = []
    
    while stack:
        vertex = stack.pop()
        if vertex not in visited:
            scc = []
            explore(vertex, visited, transposed_graph, scc)
            scc_list.append(scc)
    return scc_list



In [4]:
# Example graph
graph = {
    'A': ['B'],
    'B': ['C', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['E'],
    'E': ['F'],
    'F': ['G'],
    'G': ['F', 'H'],
    'H': ['I'],
    'I': ['J'],
    'J': ['H']
}
sccs = find_scc(graph)
print("Strongly Connected Components:")
for scc in sccs:
    print(scc)

Strongly Connected Components:
['B', 'C', 'A']
['D']
['E']
['G', 'F']
['I', 'J', 'H']


In [5]:
graph2 = {
    'A': ['B'],
    'B': ['C'],
    'C': ['D', 'E'],
    'D': [],
    'E': ['F'],
    'F': []
}

sccs = find_scc(graph2)
print("Strongly Connected Components:")
for scc in sccs:
    print(scc)

Strongly Connected Components:
['A']
['B']
['C']
['E']
['F']
['D']


In [17]:
def explore(v, visited, adj_list, stack):
    visited.add(v)
    for neighbor in adj_list[v]:
        if neighbor not in visited:
            explore(neighbor, visited, adj_list, stack)
    stack.append(v)

def transpose(graph):
    transposed = {}
    for vertex in graph:
        transposed[vertex] = []
    for vertex in graph:
        for neighbor in graph[vertex]:
            transposed[neighbor].append(vertex)
    return transposed

def find_scc(graph):
    visited = set()
    stack = []
    for vertex in graph:
        if vertex not in visited:
            explore(vertex, visited, graph, stack)
    print("Stack is: ", stack)

    transposed_graph = transpose(graph)
    print("Transposed graph is: ", transposed_graph)

    visited.clear()
    scc_list = []

    while stack:
        vertex = stack.pop()
        print("The popped element or vertex is: ", vertex)
        if vertex not in visited:
            scc = []
            explore(vertex, visited, transposed_graph, scc)
            scc_list.append(scc)
            print("scc_list is: ", scc_list)

    return scc_list


# Example graph
graph = {
    'A': ['B'],
    'B': ['C', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['E'],
    'E': ['F'],
    'F': ['G'],
    'G': ['F', 'H'],
    'H': ['I'],
    'I': ['J'],
    'J': ['H']
}



sccs = find_scc(graph)
print("Strongly Connected Components:")
for scc in sccs:
    print(scc)


Stack is:  ['J', 'I', 'H', 'G', 'F', 'C', 'E', 'D', 'B', 'A']
Transposed graph is:  {'A': ['C'], 'B': ['A'], 'C': ['B'], 'D': ['B'], 'E': ['B', 'D'], 'F': ['C', 'E', 'G'], 'G': ['F'], 'H': ['G', 'J'], 'I': ['H'], 'J': ['I']}
The popped element or vertex is:  A
scc_list is:  [['B', 'C', 'A']]
The popped element or vertex is:  B
The popped element or vertex is:  D
scc_list is:  [['B', 'C', 'A'], ['D']]
The popped element or vertex is:  E
scc_list is:  [['B', 'C', 'A'], ['D'], ['E']]
The popped element or vertex is:  C
The popped element or vertex is:  F
scc_list is:  [['B', 'C', 'A'], ['D'], ['E'], ['G', 'F']]
The popped element or vertex is:  G
The popped element or vertex is:  H
scc_list is:  [['B', 'C', 'A'], ['D'], ['E'], ['G', 'F'], ['I', 'J', 'H']]
The popped element or vertex is:  I
The popped element or vertex is:  J
Strongly Connected Components:
['B', 'C', 'A']
['D']
['E']
['G', 'F']
['I', 'J', 'H']


### Improvements on SCC by using reverse graph components

Let G<sup>R</sup> be the graph obtained from G by reversing all the edges.

### get_sink_components(G)

```
GR and G have same SCCs
Source component of GR are sink components of G
Find sink components of G by running DFS on GR
The vertex with largest postorder in GR is in a sink SCC of G
```

In [52]:
def dfs(graph, vertex, visited, stack):
    visited[vertex] = True
    for neighbor in graph[vertex]:
        if not visited[neighbor]:
            dfs(graph, neighbor, visited, stack)
    stack.append(vertex)
    
def get_sink_components(graph):
    # Step 1: Create the reversed graph GR
    reversed_graph = {v: [] for v in graph}
    for vertex in graph:
        for neighbor in graph[vertex]:
            reversed_graph[neighbor].append(vertex)
            
    # Step 2: Run DFS on GR to determine finishing time
    visited = {v: False for v in reversed_graph}
    stack = []
    for vertex in reversed_graph:
        if not visited[vertex]:
            dfs(reversed_graph, vertex, visited, stack)
    
    # Step 3: Sort vertices in reversed order based on finishing time
    sorted_vertices = stack[::-1]
    
    # Step 4: Identify the sink components
    visited = {v: False for v in graph}
    sink_components = []
    for vertex in sorted_vertices:
        if not visited[vertex]:
            component = []
            dfs(graph, vertex, visited, component)
            sink_components.append(component)
    return sink_components
    
graph = {
    'A': ['B'],
    'B': ['C', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['E'],
    'E': ['F'],
    'F': ['G'],
    'G': ['F', 'H'],
    'H': ['I'],
    'I': ['J'],
    'J': ['H']
}
sink_components = get_sink_components(graph)
print("Sink Components of G:")
for component in sink_components:
    print(component)

Sink Components of G:
['J', 'I', 'H']
['G', 'F']
['E']
['D']
['C', 'B', 'A']


In [63]:
graph = {
    'A': ['B'],
    'B': ['C', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['E'],
    'E': ['F'],
    'F': ['G'],
    'G': ['F', 'H'],
    'H': ['I'],
    'I': ['J'],
    'J': ['H']
}
reversed_graph = {v: [] for v in graph}
for vertex in graph:
    for neighbor in graph[vertex]:
        reversed_graph[neighbor].append(vertex)
            
# Step 2: Run DFS on GR to determine finishing time
visited = {v: False for v in reversed_graph}
stack = []
for vertex in reversed_graph:
    print(vertex)
    if not visited[vertex]:
        dfs(reversed_graph, vertex, visited, stack)

A
B
C
D
E
F
G
H
I
J


In [62]:
reversed_graph

{'A': ['C'],
 'B': ['A'],
 'C': ['B'],
 'D': ['B'],
 'E': ['B', 'D'],
 'F': ['C', 'E', 'G'],
 'G': ['F'],
 'H': ['G', 'J'],
 'I': ['H'],
 'J': ['I']}

In [61]:
stack

['B', 'C', 'A', 'D', 'E', 'G', 'F', 'I', 'J', 'H']

#### SCCs(G) -- Efficient version -- Didn't implement the way it is mentioned

```
run DFS(GR)
let v have largest post number
run Explore(v)
vertices found are first SCC
Remove from G and repeat
```

In [29]:
def dfs(graph, vertex, visited, stack):
    visited[vertex] = True
    for neighbor in graph[vertex]:
        if not visited[neighbor]:
            dfs(graph, neighbor, visited, stack)
    stack.append(vertex)

def explore(graph, vertex, visited, component):
    visited[vertex] = True
    for neighbor in graph[vertex]:
        if not visited[neighbor]:
            explore(graph, neighbor, visited, component)
    component.append(vertex)

def get_largest_postorder_vertex(graph):
    visited = {v: False for v in graph}
    stack = []
    for vertex in graph:
        if not visited[vertex]:
            dfs(graph, vertex, visited, stack)
    return stack[-1]

def SCCs(graph):
    sccs = []
    
    reversed_graph = {v: [] for v in graph}
    for vertex in graph:
        for neighbor in graph[vertex]:
            reversed_graph[neighbor].append(vertex)

    visited = {v: False for v in graph}
    post_order = []
    # Step 1: Run DFS on the reversed graph GR
    for vertex in graph:
        if not visited[vertex]:
            dfs(reversed_graph, vertex, visited, post_order)

    visited = {v: False for v in graph}
    for vertex in reversed(post_order):
        if not visited[vertex]:
            component = []
            explore(graph, vertex, visited, component)
            sccs.append(component)

    return sccs

# Example usage
graph = {
    'A': ['B'],
    'B': ['C', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['E'],
    'E': ['F'],
    'F': ['G'],
    'G': ['F', 'H'],
    'H': ['I'],
    'I': ['J'],
    'J': ['H']
}

sccs = SCCs(graph)
print("Sink Components of G:")
for component in sccs:
    print(component)


Sink Components of G:
['J', 'I', 'H']
['G', 'F']
['E']
['D']
['C', 'B', 'A']


### SCCs(G)

```
Run DFS(GR)
for every vertex v in reverse postorder:
    if not visited(v):
        Explore(v)
        mark visited vertices as new SCC
```

In [65]:
def dfs(graph, vertex, visited, stack):
    visited[vertex] = True
    for neighbor in graph[vertex]:
        if not visited[neighbor]:
            dfs(graph, neighbor, visited, stack)
    stack.append(vertex)
    
def explore(graph, vertex, visited, component):
    visited[vertex] = True
    component.append(vertex)
    for neighbor in graph[vertex]:
        if not visited[neighbor]:
            explore(graph, neighbor, visited, component)
            
def SCCs(graph):
    # Step 1: Run DFS on the reversed graph GR
    reversed_graph = {v: [] for v in graph}
    for vertex in graph:
        for neighbor in graph[vertex]:
            reversed_graph[neighbor].append(vertex)
            
    visited = {v: False for v in graph}
    stack = []
    for vertex in graph:
        if not visited[vertex]:
            dfs(reversed_graph, vertex, visited, stack)
    
    # Step 2: Explore vertices in reverse postorder
    sccs = []
    visited = {v: False for v in graph}
    for vertex in reversed(stack):
        if not visited[vertex]:
            component = []
            explore(graph, vertex, visited, component)
            sccs.append(component)
            #visited.update({v: True for v in component})
            
    return sccs

# Example usage
graph = {
    'A': ['B'],
    'B': ['C', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['E'],
    'E': ['F'],
    'F': ['G'],
    'G': ['F', 'H'],
    'H': ['I'],
    'I': ['J'],
    'J': ['H']
}

sccs = SCCs(graph)
print("Strongly Connected Components:")
for component in sccs:
    print(component)

Strongly Connected Components:
['H', 'I', 'J']
['F', 'G']
['E']
['D']
['A', 'B', 'C']


### Assignment 1

Task. Check whether a given directed graph with 𝑛 vertices and 𝑚 edges contains a cycle. Output 1 if the graph contains a cycle and 0 otherwise. 

```
Input:
4 4
1 2
4 1
2 3
3 1
Output:
1

Input:
5 7
1 2
2 3
1 3
3 4
1 4
2 5
3 5

Output:
0
```

In [72]:
def explore(adj, v, visited, rec_stack):
    visited[v] = True
    rec_stack[v] = True
    for neighbor in adj[v]:
        if not visited[neighbor] and explore(adj, neighbor, visited, rec_stack):
            return True
        elif rec_stack[neighbor]:
            return True
    rec_stack[v] = False
    return False

def acyclic(adj):
    n = len(adj)
    visited = [False] * n
    rec_stack = [False] * n
    for v in range(n):
        if not visited[v] and explore(adj, v, visited, rec_stack):
            return 1
    return 0

if __name__ == '__main__':
    n, m = map(int, input().split())
    edges = [list(map(int, input().split())) for _ in range(m)]
    print("edges", edges)
    adj = [[] for _ in range(n)]
    for (a, b) in edges:
        adj[a - 1].append(b - 1)
    print('adj is', adj)
    print(acyclic(adj))


5 7
1 2
2 3
1 3
3 4
1 4
2 5
3 5
edges [[1, 2], [2, 3], [1, 3], [3, 4], [1, 4], [2, 5], [3, 5]]
adj is [[1, 2, 3], [2, 4], [3, 4], [], []]
0


### Assignment 2

Task. Compute a topological ordering of a given directed acyclic graph (DAG) with 𝑛 vertices and 𝑚 edges.

```
Input:
4 3
1 2
4 1
3 1
Output:
4 3 1 2

Input:
4 1
3 1
Output:
2 3 1 4

Input:
5 7
2 1
3 2
3 1
4 3
4 1
5 2
5 3
Output:
5 4 3 2 1
```

In [1]:
import sys

def dfs(adj, used, order, vertex):
    visited[vertex] = 1
    for neighbor in adj[vertex]:
        if not used[neighbor]:
            dfs(adj, visited, order, neighbor)
    order.append(vertex)

def toposort(adj):
    used = [0] * len(adj)
    order = []
    for vertex in range(len(adj)):
        if not used[vertex]:
            dfs(adj, used, order, vertex)
    return order[::-1]

if __name__ == '__main__':
    input_data = sys.stdin.read().split()
    n, m = map(int, input_data[:2])
    data = list(map(int, input_data[2:]))
    edges = list(zip(data[0:(2 * m):2], data[1:(2 * m):2]))
    adj = [[] for _ in range(n)]
    for (a, b) in edges:
        adj[a - 1].append(b - 1)
    order = toposort(adj)
    for x in order:
        print(x + 1, end=' ')


ValueError: not enough values to unpack (expected 2, got 0)